# Homework #3:

In [32]:
import pandas as pd
import numpy as np
import pymc as pm
from scipy import stats
import matplotlib.pyplot as plt
from google.colab import files
from IPython.display import display

In [33]:
# Upload the CSV file using Colab's file upload feature
uploaded = files.upload()

Saving vgchartz_2024.csv to vgchartz_2024 (8).csv


In [34]:
# Check the uploaded file names
file_names = list(uploaded.keys())
print("Uploaded file names:", file_names)

# Read the CSV file into a DataFrame
import io

df = pd.read_csv(io.StringIO(uploaded[file_names[0]].decode('utf-8')))

# Display the DataFrame
df.head()

Uploaded file names: ['vgchartz_2024 (8).csv']


AttributeError: 'Index' object has no attribute '_format_flat'

                         title console    genre       publisher  \
0           Grand Theft Auto V     PS3   Action  Rockstar Games   
1           Grand Theft Auto V     PS4   Action  Rockstar Games   
2  Grand Theft Auto: Vice City     PS2   Action  Rockstar Games   
3           Grand Theft Auto V    X360   Action  Rockstar Games   
4    Call of Duty: Black Ops 3     PS4  Shooter      Activision   

        developer  critic_score  total_sales  na_sales  jp_sales  pal_sales  \
0  Rockstar North           9.4        20.32      6.37      0.99       9.85   
1  Rockstar North           9.7        19.39      6.06      0.60       9.71   
2  Rockstar North           9.6        16.15      8.41      0.47       5.49   
3  Rockstar North           NaN        15.86      9.06      0.06       5.33   
4        Treyarch           8.1        15.09      6.18      0.41       6.05   

   other_sales release_date last_update  
0         3.12    2013/9/17         NaN  
1         3.02   2014/11/18    2018/1/

In [ ]:
observed_data = df['total_sales']

with pm.Model() as normal_gamma_model:
    # Prior for the mean and precision (inverse variance)
    mu = pm.Normal('mu', mu=0, sigma=10)
    precision = pm.Gamma('precision', alpha=2, beta=2)

    # Likelihood (Normal distribution)
    likelihood = pm.Normal('likelihood', mu=mu, sigma=precision**-0.5, observed=observed_data)

    # Sampling
    trace_normal_gamma = pm.sample(100, tune=100)

# Posterior summary
pm.summary(trace_normal_gamma).round(2)

/usr/local/lib/python3.10/dist-packages/pymc/model.py:1363: ImputationWarning: Data in likelihood contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, ImputationWarning)


In [ ]:
with pm.Model() as new_model:
    # Choose an appropriate likelihood for non-normally distributed data
    # For example, you might use a Student's t-distribution
    nu = pm.Exponential('nu', 1/30)  # Degrees of freedom
    new_likelihood = pm.StudentT('new_likelihood', nu=nu, mu=mu, sd=precision**-0.5, observed=non_normal_data)

    # Sampling
    trace_new_model = pm.sample(1000, tune=1000)

# Posterior summary
pm.summary(trace_new_model).round(2)

In [ ]:
# Trace plot
pm.traceplot(trace_new_model)

# Posterior plot
pm.plot_posterior(trace_new_model)

# Gelman-Rubin diagnostic
pm.gelman_rubin(trace_new_model)

# Effective sample size
pm.effective_n(trace_new_model)
